# **Traitement Automatique du Langage Naturel (TALN) : Modèles de Langage N-gram**

**Objectif :** Comprendre le principe de la génération de texte à l'aide d'un modèle de langage basé sur les N-grammes, en se concentrant sur l'apprentissage des probabilités, l'entraînement sur un corpus, et l'influence du paramètre $N$.

## **Corpus d'Étude**

Le corpus utilisé pour l'entraînement de notre modèle est le suivant :

In [1]:
CORPUS = [
    "L'apprentissage automatique est un sous-domaine de l'intelligence artificielle qui permet aux ordinateurs d'apprendre à partir de données.",
    "Le Maroc est un pays situé en Afrique du Nord. Sa capitale est Rabat. Il possède une riche histoire.",
    "Rabat est la capitale du Maroc et est connue pour sa culture, son histoire et sa gastronomie.",
    "L'informatique est la science du traitement automatique de l'information par des machines."
]

## **1. Importation des Bibliothèques Nécessaires**

Nous importons les outils nécessaires pour la tokenisation, la manipulation des N-grammes, le comptage des fréquences et la génération aléatoire.

In [3]:
%pip install nltk

import sys
# Ajout du chemin local pour les bibliothèques installées
sys.path.insert(0, '/local_libs') 

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import defaultdict, Counter
import random
import string

Defaulting to user installation because normal site-packages is not writeable
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.0 MB/s  0:00:00
Using cached click-8.3.0-py3-none-any.whl (107 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

   ---------------------------------------- 0/4 [tqdm]
   ---------------------------------------- 0/4 [tqdm]
   ---------------------------------------- 0/4 [tqdm]
   ---------------------------------------- 0/4 [tqdm]
   ---------------------------------------- 0/4 [tqdm]
   ---------- ----------------------------- 1/4 [regex]
   -------------------- ------------------- 2/4 [click]
   -------------------- ------------------- 2/4 [click]
   -------------------- ------------

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **2. Nettoyage et Tokenisation**

Cette étape est cruciale pour préparer le texte. Elle consiste à :

1. **Nettoyer** la ponctuation en la séparant des mots.
2. **Convertir** le texte en minuscules.
3. **Tokeniser** (diviser en mots).
4. **Ajouter** les marqueurs de début (`<start>`) et de fin de phrase (`<end>`) pour modéliser les probabilités de début et de fin de phrase.

In [4]:
def preprocess_corpus(corpus):
    processed_sentences = []
    for sentence in corpus:
        # Séparation de la ponctuation pour une meilleure tokenisation
        sentence = sentence.replace("'", " '")
        for punct in string.punctuation:
            if punct != "'":
                sentence = sentence.replace(punct, ' ' + punct + ' ')

        # Tokenisation en minuscules (langue française)
        tokens = word_tokenize(sentence.lower(), language='french')

        # Ajout des marqueurs de début et de fin
        processed_sentences.append(['<start>'] + tokens + ['<end>'])
    return processed_sentences

tokenized_corpus = preprocess_corpus(CORPUS)
all_tokens = [token for sentence in tokenized_corpus for token in sentence]
vocabulary = set(all_tokens)
vocab_size = len(vocabulary)

print(f"Taille du vocabulaire (y compris <start> et <end>): {vocab_size}")
print("\nExtrait du Corpus Tokenisé :")
print(tokenized_corpus[:2])

Taille du vocabulaire (y compris <start> et <end>): 54

Extrait du Corpus Tokenisé :
[['<start>', 'l', "'apprentissage", 'automatique', 'est', 'un', 'sous', '-', 'domaine', 'de', 'l', "'intelligence", 'artificielle', 'qui', 'permet', 'aux', 'ordinateurs', 'd', "'apprendre", 'à', 'partir', 'de', 'données', '.', '<end>'], ['<start>', 'le', 'maroc', 'est', 'un', 'pays', 'situé', 'en', 'afrique', 'du', 'nord', '.', 'sa', 'capitale', 'est', 'rabat', '.', 'il', 'possède', 'une', 'riche', 'histoire', '.', '<end>']]


* **Analyse :**

    Ce résultat confirme la bonne exécution de l'étape de prétraitement. Le nombre 54 représente la taille totale de notre vocabulaire, $|V|$, qui est un paramètre essentiel pour le lissage de Laplace. L'extrait du corpus tokenisé montre que chaque phrase a été correctement décomposée en mots (tokens) et mise en minuscules. Surtout, la présence des marqueurs <start> et <end> est validée. Ces marqueurs sont cruciaux pour que le modèle N-gramme puisse apprendre la probabilité qu'un mot soit le premier ou le dernier d'une phrase.

## **3. Comptage des N-grammes**

Cette fonction compte les occurrences de chaque N-gramme dans le corpus. Le résultat est stocké dans un dictionnaire où la clé est le **contexte** (les $N-1$ mots précédents) et la valeur est un `Counter` des mots qui suivent ce contexte.

In [5]:
def count_ngrams(tokenized_corpus, n):
    n_gram_counts = defaultdict(Counter)

    for sentence in tokenized_corpus:
        # Génération des N-grammes pour la phrase
        sentence_ngrams = list(ngrams(sentence, n))

        for n_gram in sentence_ngrams:
            context = n_gram[:-1]
            word = n_gram[-1]
            n_gram_counts[context][word] += 1

    return n_gram_counts

# Exemple de comptage pour N=2 (Bigramme)
n_gram_counts_2 = count_ngrams(tokenized_corpus, 2)

print("Exemple de Comptage Bigramme :")
print(f"Contexte ('le',): {n_gram_counts_2[('le',)]}")
print(f"Contexte ('<start>',): {n_gram_counts_2[('<start>',)]}")

Exemple de Comptage Bigramme :
Contexte ('le',): Counter({'maroc': 1})
Contexte ('<start>',): Counter({'l': 2, 'le': 1, 'rabat': 1})


**Analyse :**

Ces résultats affichent les fréquences brutes (les comptes $C(\cdot)$) des bigrammes dans notre corpus.

* Le premier point indique que le mot 'le' n'est suivi par le mot 'maroc' qu'une seule fois ($C(\text{'le'}, \text{'maroc'}) = 1$). Le compte total du contexte $C(\text{'le'})$ est donc de 1.

* Le second point montre que le marqueur de début de phrase <start> est suivi par 'l' deux fois, par 'le' une fois, et par 'rabat' une fois. Le compte total du contexte $C(\text{<start>})$ est donc de $2+1+1=4$, ce qui correspond au nombre total de phrases dans notre corpus.

Ces comptes sont la base de l'Estimation du Maximum de Vraisemblance (EMV).

## **4. Estimer la Probabilité Conditionnelle avec Lissage de Laplace**

L'estimation de la probabilité conditionnelle $P(w_{mot} | w_{contexte})$ est effectuée en utilisant l'**Estimation du Maximum de Vraisemblance (EMV)**, ajustée par le **Lissage de Laplace** (ou *Add-one smoothing*).

### **Rôle du Lissage de Laplace**

Le lissage de Laplace résout le problème de la **rareté des données** (*data sparsity*), où des séquences de mots valides peuvent ne pas apparaître dans le petit corpus d'entraînement. Sans lissage, ces séquences auraient une probabilité de 0, ce qui rendrait impossible la génération de texte les contenant.

Le lissage de Laplace réattribue une petite partie de la masse de probabilité à tous les mots du vocabulaire, y compris ceux qui n'ont jamais été vus dans un contexte donné.

**Formule de Laplace (avec $\alpha=1$) :**



$P(w_{mot} | w_{contexte}) = \frac{C(w_{contexte}, w_{mot}) + \alpha}{C(w_{contexte}) + \alpha \cdot |V|}$


Où :

- $C(\cdot)$ est le compte d'occurrence.
- $\alpha$ est le paramètre de lissage (ici $\alpha=1$).
- $|V|$ est la taille du vocabulaire.

In [6]:
def calculate_laplace_probability(n_gram_counts, context, word, vocab_size, alpha=1):
    # C(context, word)
    count_n_gram = n_gram_counts[context].get(word, 0)

    # C(context)
    count_context = sum(n_gram_counts[context].values())

    # Formule de lissage de Laplace
    probability = (count_n_gram + alpha) / (count_context + alpha * vocab_size)
    return probability

## **5. & 6. Probabilités de Transition et Initiales**

### **Probabilités de Transition (Étape 5)**

Nous calculons les probabilités de transition pour tous les contextes observés dans le corpus.

In [7]:
def get_transition_probabilities(n_gram_counts, vocab_size, alpha=1):
    transition_probs = defaultdict(dict)

    for context, word_counts in n_gram_counts.items():
        # Pour chaque contexte, on calcule la probabilité de transition vers chaque mot du vocabulaire
        for word in vocabulary:
            prob = calculate_laplace_probability(n_gram_counts, context, word, vocab_size, alpha)
            transition_probs[context][word] = prob

    return transition_probs

transition_probs_2 = get_transition_probabilities(n_gram_counts_2, vocab_size)

print("Exemple de Probabilité de Transition Bigramme (N=2) :")
print(f"P('maroc' | 'le'): {transition_probs_2[('le',)]['maroc']:.4f}")
print(f"P('est' | 'le'): {transition_probs_2[('le',)]['est']:.4f}")

Exemple de Probabilité de Transition Bigramme (N=2) :
P('maroc' | 'le'): 0.0364
P('est' | 'le'): 0.0182


**Analyse :**

Ces valeurs sont les probabilités conditionnelles calculées en appliquant le Lissage de Laplace ($\alpha=1$) sur les comptes de la cellule précédente.

* $P(\text{'maroc'} | \text{'le'})$ à 0.0364 : Cette probabilité est calculée comme $\frac{1 + 1}{1 + 54} = \frac{2}{55}$. Le numérateur inclut le compte réel (1) plus le lissage (1). Le dénominateur inclut le compte total du contexte (1) plus le lissage appliqué à tout le vocabulaire ($1 \times 54$).

* $P(\text{'est'} | \text{'le'})$ à 0.0182 : Cette probabilité est calculée comme $\frac{0 + 1}{1 + 54} = \frac{1}{55}$. Le compte réel est zéro ($C(\text{'le'}, \text{'est'}) = 0$), car ce bigramme n'a jamais été vu. Le lissage de Laplace lui attribue une probabilité non nulle, garantissant que le modèle ne se bloque pas si la génération tente de suivre 'le' par 'est'.

### **Probabilités Initiales (Étape 6)**

Le vecteur des probabilités initiales correspond à la probabilité qu'un mot $w_1$ soit le premier mot d'une phrase, soit $P(w_1 | <start>)$.

In [8]:
def get_initial_probabilities(tokenized_corpus, vocab_size, alpha=1):
    # On compte les bigrammes de la forme (<start>, w1)
    start_ngrams = count_ngrams(tokenized_corpus, 2)

    initial_probs = {}
    start_context = ('<start>',)

    # On calcule P(w | <start>) pour chaque mot du vocabulaire
    for word in vocabulary:
        prob = calculate_laplace_probability(start_ngrams, start_context, word, vocab_size, alpha)
        initial_probs[word] = prob

    return initial_probs

initial_probs_2 = get_initial_probabilities(tokenized_corpus, vocab_size)

# Exemple de probabilité initiale
print(f"P('le' | <start>): {initial_probs_2['le']:.4f}")

P('le' | <start>): 0.0345


**Analyse :**
Cette valeur est la probabilité que le mot 'le' soit le premier mot d'une phrase. Elle est calculée comme $P(\text{'le'} | \text{<start>})$ en utilisant le lissage de Laplace sur le contexte <start>.

* **Calcul :** $\frac{C(\text{<start>}, \text{'le'}) + 1}{C(\text{<start>}) + 1 \cdot |V|} = \frac{1 + 1}{4 + 54} = \frac{2}{58}$.

* Cette probabilité est utilisée pour choisir le tout premier mot de la phrase générée. Elle montre que le modèle a une légère préférence pour les mots qui ont été vus au début des phrases dans le corpus, tout en laissant une chance à tous les autres mots du vocabulaire de commencer une phrase.


| Élément | Analyse |
| --- | --- |
| **Probabilité Initiale** | C'est la probabilité que le mot `'le'` soit le premier mot d'une phrase, soit $P(\text{'le'} |
| **Calcul :** | $\frac{C(\text{<start>}, \text{'le'}) + 1}{C(\text{<start>}) + 1 \cdot |
| **Implication** | Cette probabilité est utilisée pour choisir le tout premier mot de la phrase générée. Elle est plus faible que la probabilité de `'l'` (qui serait $\frac{2+1}{58} \approx 0.0517$) car `'l'` est apparu deux fois après `<start>` dans le corpus. |

## **7. Générer un Texte via le Modèle N-gram Entraîné**

La génération de texte est un processus itératif :

1. Choisir le premier mot en fonction des probabilités initiales.
2. Pour chaque mot suivant, déterminer le contexte (les $N-1$ mots précédents).
3. Choisir le mot suivant en fonction des probabilités de transition $P(w_{n} | w_{n-N+1:n-1})$.
4. Répéter jusqu'à ce que le marqueur `<end>` soit généré ou que la longueur maximale soit atteinte.

In [9]:
def generate_text(transition_probs, initial_probs, n, max_length=20):

    generated_text = []

    # 1. Choix du premier mot (basé sur P(w | <start>))
    words, probs = zip(*initial_probs.items())
    next_word = random.choices(words, weights=probs, k=1)[0]

    if next_word == '<end>':
        return ""

    generated_text.append(next_word)

    # 2. Génération des mots suivants
    for _ in range(max_length - 1):

        # Détermination du contexte (les N-1 derniers mots)
        if n == 1:
            context = ()
        else:
            context = tuple(generated_text[-(n-1):])

        # Récupération des probabilités de transition
        current_probs = transition_probs.get(context)

        # Si le contexte n'est pas dans les probabilités pré-calculées (cas rare avec Laplace)
        if current_probs is None:
            # On doit recalculer les probabilités pour ce contexte
            n_gram_counts = count_ngrams(tokenized_corpus, n)
            all_words = list(vocabulary)
            probs = [calculate_laplace_probability(n_gram_counts, context, w, vocab_size) for w in all_words]
            next_word = random.choices(all_words, weights=probs, k=1)[0]
        else:
            words, probs = zip(*current_probs.items())
            next_word = random.choices(words, weights=probs, k=1)[0]

        if next_word == '<end>':
            break

        generated_text.append(next_word)

    # 3. Nettoyage final du texte
    final_text = " ".join(generated_text)
    final_text = final_text.replace(" ' ", "'").replace(" . ", ". ").replace(" , ", ", ").replace(" : ", ": ").replace(" ! ", "! ").replace(" ? ", "? ").replace(" ) ", ") ").replace(" ( ", " ( ")

    return final_text.capitalize()

# --- Exécution pour N=2 (Bigramme) ---
N_VALUE_2 = 2
n_gram_counts_2 = count_ngrams(tokenized_corpus, N_VALUE_2)
transition_probs_2 = get_transition_probabilities(n_gram_counts_2, vocab_size)
generated_text_2 = generate_text(transition_probs_2, initial_probs_2, N_VALUE_2)

# --- Exécution pour N=3 (Trigramme) ---
N_VALUE_3 = 3
n_gram_counts_3 = count_ngrams(tokenized_corpus, N_VALUE_3)
transition_probs_3 = get_transition_probabilities(n_gram_counts_3, vocab_size)
generated_text_3 = generate_text(transition_probs_3, initial_probs_2, N_VALUE_3)

print(f"--- Texte Généré (N=2 - Bigramme) ---")
print(generated_text_2)
print(f"\n--- Texte Généré (N=3 - Trigramme) ---")
print(generated_text_3)

--- Texte Généré (N=2 - Bigramme) ---
Et aux connue situé rabat 'information, automatique artificielle gastronomie automatique est

--- Texte Généré (N=3 - Trigramme) ---
'apprentissage sa il d par culture - gastronomie données - il afrique d histoire sous est gastronomie connue est à


**Analyse :**

Ces résultats illustrent la limitation du modèle N-gramme sur un petit corpus.

* **N=2 (Bigramme) :** Le texte est une mosaïque de paires de mots qui sont souvent grammaticalement correctes localement (par exemple, 'automatique artificielle'), mais qui n'ont aucun sens global. Le modèle ne regarde qu'un mot en arrière, ce qui lui fait perdre toute cohérence structurelle sur le long terme.

* **N=3 (Trigramme) :** Le texte est encore plus décousu. Pour un $N$ plus grand, la plupart des séquences de $N$ mots n'existent pas dans le petit corpus. Le modèle est donc contraint d'utiliser le lissage de Laplace pour la majorité des transitions. Cela rend le choix du mot suivant presque aléatoire (probabilité uniforme), ce qui résulte en un texte sans aucune fluidité.



## **8. Conclusion**

L'atelier a permis de mettre en œuvre un modèle de langage N-gramme complet, de l'étape de prétraitement à la génération de texte.

### **Influence du Paramètre $N$**

| Paramètre $N$ | Caractéristiques | Cohérence |
| --- | --- | --- |
| **$N=1$ (Unigramme)** | Ne tient compte d'aucun contexte. La génération est purement aléatoire, ne produisant que des listes de mots. | Très faible. |
| **$N=2$ (Bigramme)** | Tient compte du mot précédent. La génération produit des paires de mots plausibles, mais la cohérence à long terme est limitée. | Faible à moyenne. |
| **$N=3$ (Trigramme)** | Tient compte des deux mots précédents. La génération produit des séquences plus longues et plus cohérentes, souvent des fragments de phrases du corpus. | Moyenne à forte. |

**Observation :** Plus $N$ est grand, plus le texte généré est **cohérent** et **proche** des phrases du corpus d'entraînement. Cependant, un $N$ trop grand augmente le risque de **surapprentissage** (*overfitting*) et le problème de la **rareté des données** (nécessitant un lissage plus agressif ou des techniques plus avancées).

### **Rôle du Lissage de Laplace**

Le lissage de Laplace est essentiel pour garantir que le modèle puisse générer des séquences qui n'étaient pas présentes dans le corpus d'entraînement. En attribuant une probabilité non nulle à tous les N-grammes possibles, il permet au modèle de généraliser et d'éviter les probabilités de transition nulles, ce qui bloquerait la génération.

Ce modèle, bien que simple, illustre parfaitement les fondements de la modélisation du langage et les défis liés à la rareté des données.